# Factorizations and other fun
Based on work by Andreas Noack

## Outline
 - Factorizations
 - Special matrix structures
 - Generic linear algebra

Before we get started, let's set up a linear system

In [3]:
using LinearAlgebra
A = rand(3, 3)
x = fill(1, (3,))
b = A * x

3-element Array{Float64,1}:
 2.1310208288918426
 2.4197178066163967
 1.0642437242746767

## Factorizations

#### LU factorizations
In Julia we can perform an LU factorization
```julia
PA = LU
``` 
where `P` is a permutation matrix, `L` is lower triangular unit diagonal and `U` is upper triangular, using `lufact`.

Julia allows computing the LU factorization and defines a composite factorization type for storing it.

In [5]:
Alu = factorize(A)

LU{Float64,Array{Float64,2}}
L factor:
3×3 Array{Float64,2}:
 1.0       0.0         0.0
 0.601772  1.0         0.0
 0.969144  0.00698348  1.0
U factor:
3×3 Array{Float64,2}:
 0.991664   0.516836  0.622521  
 0.0       -0.22719   0.00904589
 0.0        0.0       0.355975  

In [6]:
typeof(Alu)

LU{Float64,Array{Float64,2}}

#### Coming in 1.0: lufact -> lu

*In Julia 0.6, we have a family of factorization functions with a common naming scheme: `lufact`, `qrfact`, `svdfact`, etc. In 1.0, the names of these factorizations have been simplified by removing the suffixes, "fact". Instead, functions like `lu`, `qr`, and `svd` will return factorization objects.*

The different parts of the factorization can be extracted by special indexing

In [21]:
Alu.P

3×3 Array{Float64,2}:
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0

In [20]:
Alu.L

3×3 Array{Float64,2}:
 1.0       0.0         0.0
 0.601772  1.0         0.0
 0.969144  0.00698348  1.0

In [19]:
Alu.U

3×3 Array{Float64,2}:
 0.991664   0.516836  0.622521  
 0.0       -0.22719   0.00904589
 0.0        0.0       0.355975  

#### Coming in 1.0: accessing factorization components

*In 1.0, the syntax for accessing the components of a factorization has changed. To access the lower triangular matrix stored in the factorization object `Alu`, you'll write*

```julia
Alu.L
```

*instead of*

```julia
Alu[:L]
```

Julia can dispatch methods on factorization objects.

For example, we can solve the linear system using either the original matrix or the factorization object.

In [22]:
A\b

3-element Array{Float64,1}:
 0.9999999999999997
 0.999999999999999 
 1.0000000000000013

In [23]:
Alu\b

3-element Array{Float64,1}:
 0.9999999999999997
 0.999999999999999 
 1.0000000000000013

Similarly, we can calculate the determinant of `A` using either `A` or the factorization object

In [24]:
det(A) ≈ det(Alu)

true

#### QR factorizations

In Julia we can perform a QR factorization
```
A=QR
``` 

where `Q` is unitary/orthogonal and `R` is upper triangular, using `qrfact`. 

In [26]:
Aqr = qr(A) # in 1.0, qr(A)

LinearAlgebra.QRCompactWY{Float64,Array{Float64,2}}
Q factor:
3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.659184  -0.28866   -0.694371  
 -0.638845  -0.27213    0.719599  
 -0.396678   0.917944  -0.00502531
R factor:
3×3 Array{Float64,2}:
 -1.50438  -0.692918  -1.17542  
  0.0      -0.208116  -0.0885849
  0.0       0.0        0.256159 

The matrices `Q` and `R` can be extracted from the QR factorization object via

In [27]:
Aqr.Q # in 1.0, Aqr.Q

3×3 LinearAlgebra.QRCompactWYQ{Float64,Array{Float64,2}}:
 -0.659184  -0.28866   -0.694371  
 -0.638845  -0.27213    0.719599  
 -0.396678   0.917944  -0.00502531

In [28]:
Aqr.R # in 1.0, Aqr.R

3×3 Array{Float64,2}:
 -1.50438  -0.692918  -1.17542  
  0.0      -0.208116  -0.0885849
  0.0       0.0        0.256159 

#### Eigendecompositions

The results from eigendecompositions, singular value decompositions, Hessenberg factorizations, and Schur decompositions are all stored in `Factorization` types.

The eigendecomposition can be computed

In [33]:
Asym = A + A'
AsymEig = eigen(Asym) # in 1.0 eig(Asym)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
3-element Array{Float64,1}:
 -0.18054147664491177
  0.056569926880705  
  3.8732264500590587 
eigenvectors:
3×3 Array{Float64,2}:
  0.175042   0.680167  -0.711851
 -0.766294  -0.359849  -0.532261
  0.618186  -0.638656  -0.45822 

The values and the vectors can be extracted from the Eigen type by special indexing

In [35]:
AsymEig.values # in 1.0, AsymEig.values

3-element Array{Float64,1}:
 -0.18054147664491177
  0.056569926880705  
  3.8732264500590587 

In [34]:
AsymEig.vectors # in 1.0, AsymEig.vectors

3×3 Array{Float64,2}:
  0.175042   0.680167  -0.711851
 -0.766294  -0.359849  -0.532261
  0.618186  -0.638656  -0.45822 

Once again, when the factorization is stored in a type, we can dispatch on it and write specialized methods that exploit the properties of the factorization, e.g. that $A^{-1}=(V\Lambda V^{-1})^{-1}=V\Lambda^{-1}V^{-1}$.

In [36]:
inv(AsymEig)*Asym

3×3 Array{Float64,2}:
 1.0          -1.24345e-14  -7.10543e-15
 1.06581e-14   1.0           5.32907e-15
 6.21725e-15   7.10543e-15   1.0        

## Special matrix structures
Matrix structure is very important in linear algebra. To see *how* important it is, let's work with a larger linear system

In [37]:
n = 1000
A = randn(n,n);

Julia can often infer special matrix structure

In [38]:
Asym = A + A'
issymmetric(Asym)

true

but sometimes floating point error might get in the way.

In [39]:
Asym_noisy = copy(Asym)
Asym_noisy[1,2] += 5eps()

-0.3512910794688561

In [40]:
issymmetric(Asym_noisy)

false

Luckily we can declare structure explicitly with, for example, `Diagonal`, `Triangular`, `Symmetric`, `Hermitian`, `Tridiagonal` and `SymTridiagonal`.

In [41]:
Asym_explicit = Symmetric(Asym_noisy);

Let's compare how long it takes Julia to compute the eigenvalues of `Asym`, `Asym_noisy`, and `Asym_explicit`

In [42]:
@time eigvals(Asym);

  0.213250 seconds (112.26 k allocations: 13.370 MiB)


In [43]:
@time eigvals(Asym_noisy);

  1.447594 seconds (18 allocations: 7.921 MiB)


In [44]:
@time eigvals(Asym_explicit);

  0.158602 seconds (4.24 k allocations: 8.202 MiB)


In this example, using `Symmetric()` on `Asym_noisy` made our calculations about `5x` more efficient :)

#### A big problem
Using the `Tridiagonal` and `SymTridiagonal` types to store tridiagonal matrices makes it possible to work with potentially very large tridiagonal problems. The following problem would not be possible to solve on a laptop if the matrix had to be stored as a (dense) `Matrix` type.

In [45]:
n = 1_000_000;
A = SymTridiagonal(randn(n), randn(n-1));
@time eigmax(A)

  0.921881 seconds (480.05 k allocations: 206.567 MiB, 17.92% gc time)


6.701562582708217

## Generic linear algebra
The usual way of adding support for numerical linear algebra is by wrapping BLAS and LAPACK subroutines. For matrices with elements of `Float32`, `Float64`, `Complex{Float32}` or `Complex{Float64}` this is also what Julia does.

However, Julia also supports generic linear algebra, allowing you to, for example, work with matrices and vectors of rational numbers.

#### Rational numbers
Julia has rational numbers built in. To construct a rational number, use double forward slashes:

In [46]:
1//2

1//2

#### Example: Rational linear system of equations
The following example shows how linear system of equations with rational elements can be solved without promoting to floating point element types. Overflow can easily become a problem when working with rational numbers so we use `BigInt`s.

In [47]:
Arational = Matrix{Rational{BigInt}}(rand(1:10, 3, 3))/10

3×3 Array{Rational{BigInt},2}:
 1//1   1//5   3//5 
 2//5   9//10  1//1 
 7//10  1//1   9//10

In [48]:
x = fill(1, 3)
b = Arational*x

3-element Array{Rational{BigInt},1}:
  9//5 
 23//10
 13//5 

In [49]:
Arational\b

3-element Array{Rational{BigInt},1}:
 1//1
 1//1
 1//1

In [51]:
lu(Arational) # in 1.0, lu(Arational)

LU{Rational{BigInt},Array{Rational{BigInt},2}}
L factor:
3×3 Array{Rational{BigInt},2}:
 1//1    0//1   0//1
 2//5    1//1   0//1
 7//10  43//41  1//1
U factor:
3×3 Array{Rational{BigInt},2}:
 1//1   1//5     3//5 
 0//1  41//50   19//25
 0//1   0//1   -13//41

### Exercises

#### 11.1
What are the eigenvalues of matrix A?

```
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
```

In [4]:
using LinearAlgebra
A =
[
 140   97   74  168  131
  97  106   89  131   36
  74   89  152  144   71
 168  131  144   54  142
 131   36   71  142   36
]
eigenA = eigen(A)

Eigen{Float64,Float64,Array{Float64,2},Array{Float64,1}}
eigenvalues:
5-element Array{Float64,1}:
 -128.49322764802145 
  -55.887784553057   
   42.752167279318854
   87.16111477514494 
  542.4677301466137  
eigenvectors:
5×5 Array{Float64,2}:
 -0.147575  -0.647178  -0.010882    0.548903   -0.507907
 -0.256795   0.173068  -0.834628   -0.239864   -0.387253
 -0.185537  -0.239762   0.422161   -0.731925   -0.440631
  0.819704   0.247506   0.0273194   0.0366447  -0.514526
 -0.453805   0.657619   0.352577    0.322668   -0.364928

#### 11.2 
Create a `Diagonal` matrix from the eigenvalues of `A`.

In [6]:
diagm(0 => eigenA.values)

5×5 Array{Float64,2}:
 -128.493    0.0      0.0      0.0       0.0  
    0.0    -55.8878   0.0      0.0       0.0  
    0.0      0.0     42.7522   0.0       0.0  
    0.0      0.0      0.0     87.1611    0.0  
    0.0      0.0      0.0      0.0     542.468

#### 11.3 
Create a `LowerTriangular` matrix from `A`.

In [7]:
LowerTriangular(A)

5×5 LowerTriangular{Int64,Array{Int64,2}}:
 140    ⋅    ⋅    ⋅   ⋅
  97  106    ⋅    ⋅   ⋅
  74   89  152    ⋅   ⋅
 168  131  144   54   ⋅
 131   36   71  142  36

### Please let us know how we're doing!
https://tinyurl.com/introJuliaFeedback